<h1 style="background-color:green;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
    🌐 תהליך הסבת חלקות שומא לבנק"ל מודרני 🌐

In [ ]:
print(' Tax Assessment Parcels:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from configs import CNFG
import numpy as np
import pandas as pd
import geopandas as gpd
import dask_geopandas as dask_gpd
import subprocess

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('   Reading inputs')

<h1 style="text-align:right;font-size:100%">
שכבת חלקות שומא (לאחר עיבוד מקדים)

In [ ]:
parcels_shuma_cols = ['PARCEL', 'GUSH_NUM', 'GUSH_SUFFIX', 'LEGAL_AREA', 'geometry']

parcels_shuma = dask_gpd.read_file(CNFG.CADSDE + 'PARCEL_ALL_SHUMA.gdb', layer = 'PARCEL_ALL_SHUMA_remains', columns = parcels_shuma_cols, npartitions = 3).compute()

<h1 style="text-align:right;font-size:100%">
שכבת חלקות שומא מבוטלות (לאחר עיבוד מקדים)

In [ ]:
PARCEL_ALL_SHUMA_cancel_cols = ['PARCEL', 'GUSH_NUM', 'GUSH_SUFFIX', 'LEGAL_AREA', 'geometry']

parcels_shuma_cancel = dask_gpd.read_file(CNFG.CADSDE + 'PARCEL_ALL_SHUMA.gdb', layer = 'PARCEL_ALL_SHUMA_cancel', columns = parcels_shuma_cols, npartitions = 3).compute() 

<h1 style="text-align:right;font-size:100%">
נתוני גושי שומא לאחר הסבה

In [ ]:
TaxAssasmentBlocks = pd.read_csv(CNFG.CSVPath + 'BlockUniqueIDs.csv', usecols = ['Name', 'BlockUniqueID', 'GlobalID', 'IsTax'])
TaxAssasmentBlocks = TaxAssasmentBlocks[TaxAssasmentBlocks['IsTax'] == 1]
TaxAssasmentBlocks.drop(columns = 'IsTax', inplace = True)

In [ ]:
del [parcels_shuma_cols, PARCEL_ALL_SHUMA_cancel_cols]

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('   Cleaning data')

<h1 style="text-align:right;font-size:100%">
הסרת חלקות טעות שקיבלו מספר 0

In [ ]:
parcels_shuma = parcels_shuma[parcels_shuma['PARCEL'] != 0]
parcels_shuma_cancel = parcels_shuma_cancel[parcels_shuma_cancel['PARCEL'] != 0]

In [ ]:
print('   Data conversion')

<h1 style="text-align:right;font-size:100%">
הגדרת שדות של תהליכים יוצרים ומבטלים

In [ ]:
parcels_shuma_cancel['CreatedByRecord'] = CNFG.FabricatedCreateProcessGUID
parcels_shuma_cancel['RetiredByRecord'] = CNFG.FabricatedCancelProcessGUID
parcels_shuma_cancel['CreateProcessType'] = 16
parcels_shuma_cancel['CancelProcessType'] = 16


parcels_shuma['CreatedByRecord'] = CNFG.FabricatedCreateProcessGUID
parcels_shuma['CreateProcessType'] = 16

<h1 style="text-align:right;font-size:100%">
מיזוג חלקות שומא מבוטלות עם רציפות

In [ ]:
parcels_shuma = pd.concat([parcels_shuma, parcels_shuma_cancel], ignore_index = True)

<h1 style="text-align:right;font-size:100%">
הגדרת שדה מספר מלא של גוש

In [ ]:
parcels_shuma['GUSH_NUM'] = parcels_shuma['GUSH_NUM'].astype(int)

parcels_shuma['GUSH_SUFFIX'] = parcels_shuma['GUSH_SUFFIX'].fillna(0)
parcels_shuma['GUSH_SUFFIX'] = parcels_shuma['GUSH_SUFFIX'].astype(int)

parcels_shuma['BlockName'] = parcels_shuma['GUSH_NUM'].astype(str) + '/' + parcels_shuma['GUSH_SUFFIX'].astype(str)
parcels_shuma.rename(columns = {'GUSH_NUM' : 'BlockNumber', 'GUSH_SUFFIX' : 'SubBlockNumber'}, inplace = True)

<h1 style="text-align:right;font-size:100%;">
מיזוג חלקות שומא עם נתוני גושי שומא לקבלת מספר זיהוי של גוש השומא

In [ ]:
parcels_shuma = parcels_shuma.merge(TaxAssasmentBlocks, left_on = 'BlockName', right_on = 'Name', how = 'left')
parcels_shuma.rename(columns = {'GlobalID' : 'BlockGlobalID'}, inplace = True)
del TaxAssasmentBlocks

<h1 style="text-align:right;font-size:100%">
הגדרת מספר מלא שלא חלקה

In [ ]:
parcels_shuma['Name'] = parcels_shuma['PARCEL'].astype(str) + '/' + parcels_shuma['BlockName']
parcels_shuma.drop(columns = 'BlockName', inplace = True)

<h1 style="text-align:right;font-size:100%">
הסרת כפילויות של חלקות שומא, השארת הגיאומטריות שהוגדרו כשארית

In [ ]:
parcels_shuma = parcels_shuma.sort_values(['Name', 'RetiredByRecord'], na_position = 'first')
parcels_shuma = parcels_shuma.drop_duplicates(['Name'])

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [ ]:
print('   Defining new Layer')

In [ ]:
new_columns = np.dtype([
                        ("geometry", gpd.array.GeometryDtype),
                        ("Name", str),
                        ("BlockNumber", int),
                        ("SubBlockNumber", int),
                        ("StatedArea", float),
                        ("ParcelNumber", int),
                        ("BlockUniqueID", str),
                        ("LandType", int),
                        ("ParcelType", int),
                        ("StatedAreaUnit", int),
                        ("LandDesignationPlan", str),
                        ("CreatedByRecord", str),
                        ("UpdatedByRecord", str),
                        ("RetiredByRecord", str),
                        ("IsTax", int),
                        ("CreateProcessType", int),
                        ("CancelProcessType", int)
                        ])


TaxAssessmentParcels = gpd.GeoDataFrame(data = np.empty(0, dtype=new_columns), crs = 2039, geometry = 'geometry')

del new_columns

<h1 style="text-align:right;font-size:100%">
איכלוס שדות

In [ ]:
TaxAssessmentParcels['ParcelNumber'] = parcels_shuma['PARCEL'].astype(int)

TaxAssessmentParcels['BlockNumber'] = parcels_shuma['BlockNumber'].astype(int)

TaxAssessmentParcels['SubBlockNumber'] = 0

TaxAssessmentParcels['Name'] = parcels_shuma['Name']

TaxAssessmentParcels['CreatedByRecord'] = parcels_shuma['CreatedByRecord']

TaxAssessmentParcels['RetiredByRecord'] = parcels_shuma['RetiredByRecord']

TaxAssessmentParcels['StatedAreaUnit'] = 109404 #Square Meter

TaxAssessmentParcels['ParcelType'] = 2 # סופית

TaxAssessmentParcels['LandType']   = 2 # לא מוסדר

TaxAssessmentParcels['StatedArea'] = parcels_shuma['LEGAL_AREA']

TaxAssessmentParcels['BlockUniqueID'] = parcels_shuma['BlockGlobalID']

TaxAssessmentParcels['CreateProcessType'] = parcels_shuma['CreateProcessType']

TaxAssessmentParcels['CancelProcessType'] = parcels_shuma['CancelProcessType'].astype('Int64')

TaxAssessmentParcels['geometry'] = parcels_shuma['geometry']

TaxAssessmentParcels.sort_values(by = ['BlockNumber', 'ParcelNumber'], inplace = True)

TaxAssessmentParcels['IsTax'] = 1   #שומא

In [ ]:
del parcels_shuma

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾

In [ ]:
print('   Exporting results')

In [ ]:
TaxAssessmentParcels.to_file(CNFG.Results, layer = 'TaxParcels', driver = 'OpenFileGDB', engine = 'pyogrio', mode = 'w')

In [ ]:
print('   Appending results to Parcels2D')

In [ ]:
subprocess.call([CNFG.BatchPath + "Parcels2D(current)-Save as Feature Class.bat"]);